In [31]:
from glob import glob
import pydicom as dicom
import numpy as np
import os
import matplotlib.pyplot as plt
from point import Point
from vector import Vector
from ray import Ray

In [27]:
input_dir = 'dataset/patient_A'
series = os.listdir(input_dir)
files = glob(f"{input_dir}/{series[1]}/*.dcm")

data = {}
locations = []
for file in files:
    img = dicom.dcmread(file)
    loc = float(img.SliceLocation)
    locations.append(loc)
    data[loc] = img.pixel_array
slices = []
locations.sort()
for location in locations:
    slices.append(data[location])
img = np.stack(slices)
img = img/img.max()
img = np.transpose(img,[1,2,0])

In [32]:
width = 300
height = 200
aspect_ratio = float(width) / height
x0 = -1.0
x1 = +1.0
xstep = (x1 - x0) / (width - 1)
y0 = -1.0 / aspect_ratio
y1 = +1.0 / aspect_ratio
ystep = (y1 - y0) / (height - 1)

camera = Vector(0,-35,-1)
pixels = np.zeros((width,height))

prog = 0.0
cur_prog = prog
total = width * height
bar = 1
print('Rendering:# {}%'.format(cur_prog),end='\r')
# Lets throw some rays
for j in range(height):
    y = y0 + j * ystep
    for i in range(width):
        x = x0 + i * xstep
        ray = Ray(camera, Point(x,y) - camera)
        break
    break
#         pixels.set_pixel(i,j, self.ray_trace(ray, scene))
#         prog += 1
#         if (prog/total*100) >= (cur_prog/total*100 + 5):
#             cur_prog = prog
#             bar += 1
#             print('Rendering:{} {}%'.format(''.join(['#' for x in range(bar)]),int(cur_prog/total*100)),end='\r')
# print('Rendering:{} {}%'.format(''.join(['#' for x in range(bar+1)]),100),end='\n')

In [42]:
print(ray.direction)

(-0.02910153616513078, 0.9991527416694902, 0.02910153616513078)


In [ ]:
v_orgin = [0,0]
v = np.array([[0,0,0,0,0,0],
              [0,1,0,1,1,0],
              [0,1,0,1,1,0],
              [0,0,0,0,0,0]])

camera = [0,-2]
width = 6
camera_direction = camera - v_orgin
l_orgin = 

for i in width:


In [46]:
v.shape

(4, 6)

In [ ]:

def Ray(orgin,direction):
    orgin = orgin
    direction = 